# Segmenting and Clustering Neighborhoods in Toronto

In this notebook we are going to explore and cluster the neighborhoods in Toronto

## 1. Data Scraping and creating dataframe

import BeautifulSoup for web scraping and other required libraries

In [1]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

Scrap webtable data and convert it into pandas dataframe

In [41]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs(page.content, 'html.parser')
table = soup.find('table', class_='wikitable sortable')
cols = [th.get_text().rstrip('\n') for th in table.find_all('th')]
rows = [[td.get_text().rstrip('\n') for td in tr.find_all('td')] for tr in table.find_all('tr')[1:]]
df = pd.DataFrame(data = rows, columns = cols)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean the data:

In [42]:
df.drop(df[df['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)

def func(data_row):
    if data_row['Neighbourhood'] == 'Not assigned' :
        return data_row['Borough']
    else:
        return data_row['Neighbourhood']
df['Neighbourhood'] = df.apply(func, axis=1)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [82]:
def func1(data):
    neighbours = ''
    for d in data:
        if(neighbours != ''):
            neighbours = neighbours + ',' + d
        else:
            neighbours = d
    return neighbours
def func2(data):
    for d in data:
        return d
df1 = df.groupby('Postcode').agg({'Borough': func2,
                                 'Neighbourhood':func1}, axis = 1)
df1

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
